## Установка библиотек

In [1]:
%%capture
!pip install langchain langchain_community langchain-ollama

## Установка Ollama

In [2]:
%%capture
!curl https://ollama.ai/install.sh | sh

## Импорты

In [3]:
import math
import os
import subprocess
import threading
from math import pi

import pandas as pd
from langchain_ollama.llms import OllamaLLM
from tqdm import tqdm

from langchain.agents import AgentType, initialize_agent, tool
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

seed = 42

## Запуск Ollama и инициализация модели

In [4]:
def ollama():
    os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
    os.environ["OLLAMA_ORIGINS"] = "*"
    subprocess.Popen(["ollama", "serve"])

In [5]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

В качестве LLM будем использовать **qwen2.5** на 14 миллиардов параметров с квантизацией.

In [ ]:
!ollama run qwen2.5:14b

In [7]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [8]:
model = OllamaLLM(model="qwen2.5:14b")

## Парсинг данных

In [9]:
df = pd.read_csv(
    "https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv"
)

Зададим текст промта с указанием необходимых инструкций.  
С целью повышения качества ответов модели укажем, что ответ очень важен для нас, а также назначим чаевые.

In [10]:
template = """Из следующего текста извлеки информацию:

job_title: Название вакансии без указания грейда
(например, Senior Python Developer -> Python Developer).
Если нет или неизвестно, верни null.

company: Название компании, как указано в описании, без дополнительных описаний
(например, "финтех", "крупная компания").
Если нет или неизвестно, верни null.

salary: Значение зарплаты:
- Числа без пробелов. Если указано "тыс." или "к.", умножай на 1000.
- После числа указывай валюту (руб. или $).
- Диапазоны пишутся через тире без пробелов (например, 100000-150000 руб.).
- Если указана только нижняя граница, добавляй "от" перед числом
(например, от 100000 руб.).
- Если указана только верхняя граница, добавляй "до" перед числом
(например, до 150000 руб.).
- Если зарплата указана за час, добавляй "в час" в конце.
- Исключай информацию о бонусах, премиях, процентах,
ДМС и других дополнительных выплатах.
Если нет или неизвестно, верни null.

tg: Контакт в Telegram. Указывай с @. Если контактов несколько,
перечисляй через запятую с пробелом после запятой. Если нет или неизвестно, верни null.

grade: Грейд вакансии (возможные значения: intern, junior, junior+, middle, middle+, senior, lead).
Если указано несколько, перечисляй через запятую в порядке возрастания с пробелом после запятой.
Если нет или неизвестно, верни null.

Строго следуй переданным инструкциям! От этого зависят несколько жизней!
Если выполнишь задачу на отлично, то получишь чаевые 10$!

###

Текст
text: {text}

###

{format_instructions}
"""

prompt = PromptTemplate.from_template(template=template)

Зададим схемы для парсера выхода.

In [11]:
job_title_schema = ResponseSchema(
    name="job_title",
    description="Название вакансии без грейда. Если нет или неизвестно, верни null.",
)

company_schema = ResponseSchema(
    name="company",
    description="Название компании. Если нет или неизвестно, верни null.",
)

salary_schema = ResponseSchema(
    name="salary",
    description="""
    Значение зарплаты:
    - Числа без пробелов. Если указано "тыс." или "к.", умножай на 1000.
    - После числа указывай валюту (руб. или $).
    - Диапазоны пишутся через тире без пробелов.
    - Если указана только нижняя граница, добавляй "от" перед числом.
    - Если указана только верхняя граница, добавляй "до" перед числом.
    - Если зарплата указана за час, добавляй "в час" в конце.
    - Исключай информацию о бонусах, премиях, процентах и других дополнительных выплатах.
    Если нет или неизвестно, верни null.
    """,
)

tg_schema = ResponseSchema(
    name="tg",
    description="""Контакт в Telegram. Указывай с @.
    Если контактов несколько, перечисляй через запятую с
    пробелом после запятой. Если нет или неизвестно, верни null.
    """,
)

grade_schema = ResponseSchema(
    name="grade",
    description="""Грейд вакансии.
    Если указано несколько, перечисляй через запятую в порядке возрастания.
    Если нет или неизвестно, верни null.
    """,
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema,
]

Проинициализируем парсер и получим инструкции для выхода модели.

In [12]:
output_parser = StructuredOutputParser.from_response_schemas(
    response_schemas
)
format_instructions = (
    output_parser.get_format_instructions()
)

Зададим словарь для обработки некорректных выводов модели.  
При возникновении ошибок все поля будем заполнять *None*.

In [13]:
empty_dict = {
    "job_title": None,
    "company": None,
    "salary": None,
    "tg": None,
    "grade": None,
}

Выполним парсинг данных при помощи LLM.

In [14]:
dict_list = []
for text_input in tqdm(df["text"]):
    query = prompt.format(text=text_input, format_instructions=format_instructions)
    response = model.invoke(query)
    try:
        dict_list.append(output_parser.parse(response))
    except:
        dict_list.append(empty_dict)

100%|██████████| 50/50 [05:44<00:00,  6.90s/it]


Соберем датафрейм с исходными данными и результатами парсинга.

In [15]:
result_df = pd.DataFrame(dict_list)
ans_df = pd.concat([df, result_df], axis=1)

In [16]:
result_cols = result_df.columns

## Анализ результатов парсинга

In [17]:
pd.set_option('display.max_colwidth', None)

In [18]:
vac_res_sample = ans_df.sample(5, random_state=seed).reset_index(drop=True)

In [19]:
print(vac_res_sample.loc[0, 'text'])

#job #вакансия #работа #fulltime  #TS #Typescript #vue #frontend #удаленка #senior

Вакансия: Frontend разработчик
Компания: Travelata
Город: Москва
Вилка: 280к-320к.net
Формат: удаленный

О компании:
Travelata — крупная туристическая компания с международными инвестициями более 10 млн. долларов. Входит в группу компаний, работающих в 5 странах Европы, и является членом Альянса Туристических Агентств.

Компания “Travelata” ищет senior frontend разработчика с опытом от 5 лет.
 
🔸Технологический стек и используемые инструменты: Vue.js, Vuex, Typescript, Webpack, Backbone.js, Backbone.Epoxy, Angular.js, Bootstrap, jQuery,   jQuery Mobile, underscore.js.
 
🔸Вам предстоит:
-Проектировать и разрабатывать новые компоненты портала по поиску туров, рефакторить и дорабатывать существующие компоненты;
-Верстать макеты, экспортированные из Sketch в Zeplin;
-Писать unit-тесты на Jest;
-Находить решения сложных задач и уметь самостоятельно превратить бизнес-идею в поддерживаемый код;
-Активно участв

In [20]:
vac_res_sample.loc[0, result_cols]

,0
job_title,Frontend разработчик
company,Travelata
salary,280000-320000 руб.
tg,@ann_gfio
grade,senior


Модель верно спарсила все необходимые поля.

In [21]:
print(vac_res_sample.loc[1, 'text'])

#copywriter #вакансия  #fulltime #удаленка  #remote #crypto #blockchain

Компания: ALEX GROUP COMPANY
Вилка: от 80 000 до 140 000 руб.
Вакансия: Копирайтер в сфере крипты

Мы в AlexGroup специализируемся на создании и переводе контента носителями на криптовалютную тематику и многие другие. 

Ищем копирайтера (менеджер в TG, linkedin, reddit) в крипто сфере

Требования:
— Уверенное владение английским языком: разговорный и письменный;
— Опыт работы с крипто, блокчейн и финтех проектами обязателен;
— Написание уникальных текстов  SEO-оптимизация, форматированные заголовками, абзацами, списками.;
— Составление официальных документов (питч-деки, WP, прочее).
— Подготовка любого рода текстовых материалов: статей, в том числе аналитических, пресс-релизов, SMM текстов, продающих текстов, интервью и так дале

Что мы предлагаем:
— Fulltime-работа. Возможен гибкий график (с сохранением продуктивности)
— Конкурентная ЗП от 80 000 до 140 000 р., подробнее обговорим лично
— Работа в одной из топовы

In [22]:
vac_res_sample.loc[1, result_cols]

,1
job_title,Копирайтер
company,ALEX GROUP COMPANY
salary,от 80000 до 140000 руб.
tg,@danilavaren
grade,None


Модель справилась со всеми полями кроме зарплаты.  
Модель вывела зарплату не через тире.

In [23]:
print(vac_res_sample.loc[2, 'text'])

#junior #vacancy #вакансия #recruiter

Junior IT Recruiter
Full-time | Remote

Привет! 👋
Меня зовут Настя, я рекрутер компании AIPeople. Наша команда занимается передовыми проектами в сфере финтеха. ML и блокчейн - это наша стихия. Работаем из офисов в Петербурге, на Кипре и удаленно.
Сейчас мы ищем в нашу команду Junior IT Recruiter.
Мы готовы рассмотреть кандидатов с минимальным опытом в IT подборе, но большим желанием развиваться в этом направлении. У нас небольшая команда из 2х человек, будем рады, если ты тоже к нам присоединишься!
Спектр задач на позицию довольно стандартный. Что важнее - это твой подход к работе, самостоятельность и желание достичь результата!

Чем предстоит заниматься:
 • Снятие заявки на вакансию, формирование профиля подходящего кандидата;
 • Формирование пула кандидатов используя различные источники, первый контакт с ними;
 • Ведение кандидата по всем этапам до оффера, предоставление оперативной обратной связи;
 • Ведение базы кандидатов (у нас Huntflow);
 •

In [24]:
vac_res_sample.loc[2, result_cols]

,2
job_title,IT Recruiter
company,AIPeople
salary,None
tg,@nastyaishiring
grade,junior


Модель справилась со всеми полями.

In [25]:
print(vac_res_sample.loc[3, 'text'])

#вакансия

Middle+/Senior Backend Engineer в стартап
110 000 – 140 000 ₽/месяц
Удаленка, парттайм

Qlan  - AI стартап. Мы создаем NoCode ML платформу и нишевые сервисы для бизнеса. Наша команда - это  амбициозные и проактивные разработчики, нацеленные на изменение мира ML


🎯 Примеры будущих задач:
       - Разработка инфраструктуры для наших продуктов 
       - Разработка микросервисов для аутсорс заказов


🗿Стек нашего идеального кандидата:
      - Python, Docker, Git, Flask, FastAPI, Django


🫶 Будет здорово, если вы: 
       - Имеете  крутые репозитории на github с большим количеством звездочек))))

На связи: @laptev13


In [26]:
vac_res_sample.loc[3, result_cols]

,3
job_title,Backend Engineer
company,Qlan
salary,110000-140000 руб.
tg,@laptev13
grade,"middle+, senior"


Модель справилась отлично, даже уловила два грейда.

In [27]:
print(vac_res_sample.loc[4, 'text'])

#job #вакансия #работа #fulltime #remote #python #django #удаленка  #senior 

Позиция: Senior Python developer
Компания: Skillbox
Формат: Удаленно/офис
Местоположение: Россия, Москва
Вилка: 300-400к .net 
Занятость: full-time

Приглашаем тебя стать частью команды Skillbox — ведущей образовательной онлайн-платформы в России, которая объединяет ведущих экспертов и практиков рынка. Сейчас на платформе из более чем 800 образовательных продуктов, а также корпоративным обучением для бизнеса.

В настоящий момент Skillbox активно ищет опытного Senior Python разработчика, с опытом работы не менее 5 лет, который готов присоединиться к их дружной команде, ответственной за развитие образовательной платформы.

Технический стек: Python, Django, FastAPI, Redis,  RabbitMQ, pytest, PostgreSQL.

Тебе предстоит:
- Разрабатывать платформу обучения и связанных сервисов;
- Проектировать архитектуру сервисов и их взаимодействий;
- Покрывать код тестами (юниты/функциональные/интеграционные), codereview, разби

In [28]:
vac_res_sample.loc[4, result_cols]

,4
job_title,Python developer
company,Skillbox
salary,300-400000 руб.
tg,@ann_gfio
grade,senior


Модель ошиблась при парсинге зарплаты, вместо 300 тысяч. она восприняла текст в описании как 300.

В целом, результаты парсинга можно считать успешными.  
Модель ошибается при парсинге зарплаты.  
Вероятно, явное указание примеров парсинга зарплаты в промте поможет снизить вероятность ошибок модели в данном поле.

## Математические задачки

In [29]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110884/custom_math_tools.csv")
df.head()

,task
0,Периметр прямоугольника со сторонами 1.5 м и 2 м. Отвей дай в сантиметрах.
1,Найди сумму площадей круга с радиусом 5 см и круга с радиусом 3 см.
2,Переведи числа 20 и 32 из десятичной в двоичную систему счисления и найди сумму единиц.
3,Переведи числа из двоичной системы счисления 1101011010 и 101011 и посчитай их сумму.
4,"Найди объем цилиндра с радиусом 10 см и высотой 30 см, переведи результат в литры."


Объявим необходимые инструменты.

In [30]:
@tool
def compute_arc_length(radius, angle_in_degrees):
    """Вычисляет длину дуги по радиусу и углу в градусах"""
    return (angle_in_degrees / 360) * 2 * math.pi * radius


@tool
def compute_triangle_area(a, b, c):
    """Вычисляет площадь треугольника по длинам его сторон"""
    s = (a + b + c) / 2
    return math.sqrt(s * (s - a) * (s - b) * (s - c))


@tool
def add_numbers(num1, num2):
    """Складывает два числа"""
    return num1 + num2


@tool
def convert_meters_to_cm(num):
    """Переводит метры в сантиметры"""
    return num * 100


@tool
def convert_cubic_cm_to_liters(num):
    """Переводит кубические сантиметры в литры"""
    return num / 1000


@tool
def compute_rectangle_perimeter(a, b):
    """Вычисляет периметр прямоугольника"""
    return 2 * (a + b)


@tool
def compute_circle_area(radius):
    """Вычисляет площадь круга"""
    return math.pi * radius**2


@tool
def compute_cylinder_volume(radius, height):
    """Вычисляет объем цилиндра"""
    return math.pi * radius**2 * height


@tool
def compute_cube_volume(a):
    """Вычисляет объем куба"""
    return a**3


@tool
def convert_binary_to_decimal(binary_number: str):
    """Переводит число из двоичной системы счисления в десятичную систему счисления"""
    return int(binary_number, 2)


@tool
def convert_decimal_to_binary(decimal_number):
    """Переводит число из десятичной системы счисления в двоичную систему счисления."""
    return bin(decimal_number)[2:]


@tool
def get_count_ones(number: str):
    """Находит количество единиц в двоичном представлении числа."""
    return number.count("1")

In [31]:
tools = [
    compute_arc_length,
    compute_triangle_area,
    add_numbers,
    convert_meters_to_cm,
    convert_cubic_cm_to_liters,
    compute_rectangle_perimeter,
    compute_circle_area,
    compute_cylinder_volume,
    compute_cube_volume,
    convert_binary_to_decimal,
    convert_decimal_to_binary,
    get_count_ones,
]

Проинициализируем агента и передадим ему набор инструментов.

In [41]:
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=model,
    verbose=True,
)

Решим задачки при помощи агента.

In [42]:
results = []
for task in tqdm(df["task"]):
    result = agent(task + "\n\nВ качестве ответа выдай только число.")["output"]
    results.append(result)

  0%|          | 0/10 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Для начала нужно рассчитать периметр прямоугольника в метрах, используя формулу для периметра прямоугольника (P = 2 * (a + b)), где a и b - длины сторон. Затем необходимо перевести полученное значение из метров в сантиметры.
Action:
```
{
  "action": "compute_rectangle_perimeter",
  "action_input": {
    "a": "1.5",
    "b": "2"
  }
}
```
Observation: 1.521.52
Thought:Периметр прямоугольника в метрах равен 7. Теперь нужно перевести это значение из метров в сантиметры.
Action:
```
{
  "action": "convert_meters_to_cm",
  "action_input": {
    "num": "7"
  }
}
```
Observation: 7777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777
Thought:

 10%|█         | 1/10 [00:20<03:06, 20.72s/it]

Периметр прямоугольника в сантиметрах равен 700.
Action:
```
{
  "action": "Final Answer",
  "action_input": "700"
}
```

> Finished chain.


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "compute_circle_area",
  "action_input": {
    "radius": 5
  }
}
```

Observation: 78.53981633974483
Thought:Action:
```
{
  "action": "compute_circle_area",
  "action_input": {
    "radius": 3
  }
}
```
Observation: 28.274333882308138
Thought:Action:
```
{
  "action": "add_numbers",
  "action_input": {
    "num1": 78.53981633974483,
    "num2": 28.274333882308138
  }
}
```

Observation: 106.81415022205297
Thought:

 20%|██        | 2/10 [00:33<02:06, 15.79s/it]

...

I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "1111101"
}
```

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need to convert the binary number 110010 into decimal first before computing the area of the triangle.
Action:
```
{
  "action": "convert_binary_to_decimal",
  "action_input": {
    "binary_number": "110010"
  }
}
```


Observation: 50
Thought:Теперь, когда я знаю все стороны треугольника в десятичной системе счисления (30 см, 40 см и 50 см), я могу вычислить его площадь.
Action:
```
{
  "action": "compute_triangle_area",
  "action_input": {
    "a": 30,
    "b": 40,
    "c": 50
  }
}
```
Observation: 600.0
Thought:

 90%|█████████ | 9/10 [02:12<00:12, 12.94s/it]

Я знаю, что в ответе нужно предоставить только число.
Action:
```
{
  "action": "Final Answer",
  "action_input": "600"
}
```

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Для начала нужно рассчитать объем каждого цилиндра с использованием функции compute_cylinder_volume. Затем сложить два полученных объема, чтобы найти их сумму. В конце концов, перевести эту сумму из кубических сантиметров в литры, используя функцию convert_cubic_cm_to_liters.

Action:
```
{
  "action": "compute_cylinder_volume",
  "action_input": {
    "radius": 8,
    "height": 20
  }
}
```
Observation: 4021.238596594935
Thought:Теперь нужно рассчитать объем второго цилиндра и затем сложить оба полученных значения.

Action:
```
{
  "action": "compute_cylinder_volume",
  "action_input": {
    "radius": 12,
    "height": 25
  }
}
```

Observation: 11309.733552923255
Thought:Теперь, когда у нас есть объемы обоих цилиндров, нужно сложить эти значения и перевести сумму из кубических сантиметров в ли

100%|██████████| 10/10 [02:45<00:00, 16.57s/it]

Теперь у нас есть общая сумма объемов двух цилиндров в литрах.

Action:
```
{
  "action": "Final Answer",
  "action_input": "15.33097214951819"
}
```

> Finished chain.


Внесем результаты решения задач в датафрейм.

In [43]:
df["answer"] = results
df["answer"] = df["answer"].astype(float)

## Анализ результатов решения задач

In [46]:
math_res_sample = df.sample(5, random_state=seed).reset_index(drop=True)

In [47]:
print(math_res_sample.loc[0, 'task'])

Стороны треугольника соответственно равны 30 см, 40 см и 110010 см (переведи это число из двоичной системы счисления в десятичную). Найди площадь этого треугольника.


In [48]:
math_res_sample.loc[0, 'answer']

600.0

In [69]:
compute_triangle_area.func(convert_binary_to_decimal.func("110010"), 30, 40)

600.0

Модель выдала верный результат.

In [50]:
print(math_res_sample.loc[1, 'task'])

Найди сумму площадей круга с радиусом 5 см и круга с радиусом 3 см.


In [51]:
math_res_sample.loc[1, 'answer']

106.81415022205297

In [77]:
add_numbers.func(compute_circle_area.func(5), compute_circle_area.func(3))

106.81415022205297

Результат верный.

In [53]:
print(math_res_sample.loc[2, 'task'])

Найди длину дуги окружности с радиусом 20 м, образованной центральным углом 60°, затем переведи результат в сантиметры.


In [54]:
math_res_sample.loc[2, 'answer']

2094.4

In [78]:
convert_meters_to_cm.func(compute_arc_length.func(20, 60))

2094.395102393195

Ответ модели так же верный.

In [57]:
print(math_res_sample.loc[3, 'task'])

Периметр прямоугольника со сторонами 1.5 м и 2 м. Отвей дай в сантиметрах.


In [58]:
math_res_sample.loc[3, 'answer']

700.0

In [79]:
convert_meters_to_cm.func(compute_rectangle_perimeter.func(1.5, 2))

700.0

Верно.

In [59]:
print(math_res_sample.loc[4, 'task'])

Найти объем куба со стороной 5 см, затем перевести результат в двоичную систему счисления.


In [60]:
math_res_sample.loc[4, 'answer']

1111101.0

In [80]:
convert_decimal_to_binary.func(compute_cube_volume.func(5))

'1111101'

Верно.

Таким образом, агентный подход показал неплохие результаты в решении представленных математических задач.  
Из рассмотренных ответов все оказались верными.